In [1]:
import torch
import torch.nn.functional as F
import json
import numpy as np
from espnet.utils.io_utils import LoadInputsAndTargets
from espnet.utils.training.batchfy import make_batchset
from espnet.asr.pytorch_backend.asr_init import load_trained_model
from espnet.asr.pytorch_backend.asr import CustomConverter

In [2]:
from geomloss import SamplesLoss

In [3]:
loss = SamplesLoss(p=2, blur=0.01) 

In [4]:
x = torch.rand(100,10)
y = torch.rand(100,10)

In [5]:
device = torch.device("cpu")

In [6]:
with open("/root/kws_data/dump/train/deltafalse/data.json", "r") as f:
    data_json = json.load(f)["utts"]

In [7]:
data_batches = make_batchset(data_json, 1)

In [8]:
load_tr = LoadInputsAndTargets(
        mode='asr', load_output=True, preprocess_conf=None,
        preprocess_args={'train': True}  # Switch the mode of preprocessing
    )

In [9]:
model, train_args = load_trained_model("/shreekanth2/train_clean_100_pytorch_phone_hybrid/results/model.acc.best")
model = model.to(device=device)
model.dec.sampling_probability = 0

/root/espnet/tools/venv/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [10]:
converter = CustomConverter(subsampling_factor=1, dtype=torch.float32)

In [11]:
data_batches[1000]

[('c673bc5e-859b-4d4d-ab86-ff4d7a9c9737',
  {'input': [{'feat': '/root/kws_data/dump/train/deltafalse//feats.25.ark:41105713',
     'name': 'input1',
     'shape': [741, 80]}],
   'output': [{'name': 'target1',
     'shape': [1, 5],
     'text': '0',
     'token': '0',
     'tokenid': '2'}],
   'utt2spk': 'c673bc5e-859b-4d4d-ab86-ff4d7a9c9737'})]

In [12]:
data_batches[-200]

[('38e81e02-72a7-46b5-b551-134e00df66b3',
  {'input': [{'feat': '/root/kws_data/dump/train/deltafalse//feats.8.ark:5891195',
     'name': 'input1',
     'shape': [128, 80]}],
   'output': [{'name': 'target1',
     'shape': [1, 5],
     'text': '1',
     'token': '1',
     'tokenid': '3'}],
   'utt2spk': '38e81e02-72a7-46b5-b551-134e00df66b3'})]

In [13]:
phone_to_int = dict(zip(train_args.char_list, np.arange(len(train_args.char_list))))

In [14]:
keyword = "<space> HH EY <space> S N IH P S <space>"

In [15]:
keyword_tokens = torch.tensor([[phone_to_int[phn] for phn in keyword.split(" ")]]).to(device)

In [16]:
keyword_tokens

tensor([[ 2, 18, 15,  2, 31, 25, 19, 29, 31,  2]])

In [17]:
neg_data_input = [load_tr(data_batches[1000])]
pos_data_input = [load_tr(data_batches[-200])]

In [18]:
neg_data = converter(neg_data_input, device)
pos_data = converter(pos_data_input, device)

In [21]:
# @torch.jit.script
# def x_encode(model, hs_pad, hlens):
#     return model.enc(hs_pad, hlens)

In [26]:
model.enc

Encoder(
  (enc): ModuleList(
    (0): RNNP(
      (birnn0): GRU(80, 512, batch_first=True, dropout=0.1, bidirectional=True)
      (bt0): Linear(in_features=1024, out_features=512, bias=True)
      (birnn1): GRU(512, 512, batch_first=True, dropout=0.1, bidirectional=True)
      (bt1): Linear(in_features=1024, out_features=512, bias=True)
      (birnn2): GRU(512, 512, batch_first=True, dropout=0.1, bidirectional=True)
      (bt2): Linear(in_features=1024, out_features=512, bias=True)
      (birnn3): GRU(512, 512, batch_first=True, dropout=0.1, bidirectional=True)
      (bt3): Linear(in_features=1024, out_features=512, bias=True)
      (birnn4): GRU(512, 512, batch_first=True, dropout=0.1, bidirectional=True)
      (bt4): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
)

In [ ]:
# neg_hs_pad, neg_hlens, _ = model.enc(neg_data[0], neg_data[1])
# pos_hs_pad, pos_hlens, _ = model.enc(pos_data[0], pos_data[1])

In [ ]:
# model.dec(pos_hs_pad, pos_hlens, keyword_tokens)

In [ ]:
# att_loss, acc, _ = model.dec(neg_hs_pad, neg_hlens, keyword_tokens)
# float(att_loss.cpu().detach().numpy())